In [1]:
# kd cross，close > ema 8, ema 8 > ema 18, ema 18 > 38, take profit atr, stop loss atr

In [2]:
import requests
import json
import pandas as pd
import datetime as dt
import time
import ta
import os


In [3]:
# 調整 notebook 寬物
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }<style>"))

# 調整 output 不折疊
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
import winsound
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [5]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)

     asset        free      locked
0      BTC  0.00000000  0.00000000
1      LTC  0.00000000  0.00000000
2      ETH  0.00000000  0.00000000
3      NEO  0.00000000  0.00000000
4      BNB  0.04622450  0.00000000
..     ...         ...         ...
536    EDU  0.00000000  0.00000000
537  WBETH  0.00000000  0.00000000
538    SUI  0.00000000  0.00000000
539   PEPE        0.00        0.00
540  FLOKI        0.00        0.00

[541 rows x 3 columns]


In [6]:
timezone = 8
symbol = 'ethusdt'
interval = '15m'

# start epoch till now
start_time = 1684242963000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600

In [7]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    raw_df = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol.upper(),
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        raw_df = pd.concat([raw_df, out], axis = 0)
    
    raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    raw_df = raw_df.astype(convert_dict)

    raw_df['Close_Time'] = pd.to_datetime(raw_df['Close_Time'], unit = 'ms')
    raw_df['Close_Time'] = raw_df['Close_Time'] + pd.Timedelta(hours=timezone)
    raw_df['Close_Time'] = raw_df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    raw_df = raw_df.reset_index(drop=True)
    
    return raw_df

In [8]:
raw_df = get_historical(symbol, interval, start_time, end_time, step)
print(raw_df)

              Close_Time     Open    Close     High      Low     Volume
0    2023-05-16 21:44:59  1819.40  1819.42  1821.40  1813.36  4237.8524
1    2023-05-16 21:59:59  1819.42  1814.20  1821.60  1813.84  4498.2121
2    2023-05-16 22:14:59  1814.20  1813.63  1818.14  1810.71  6753.9905
3    2023-05-16 22:29:59  1813.63  1815.70  1816.42  1812.13  2790.3098
4    2023-05-16 22:44:59  1815.69  1813.86  1817.75  1811.05  3894.4049
..                   ...      ...      ...      ...      ...        ...
385  2023-05-20 21:59:59  1815.03  1814.26  1815.49  1814.01   999.1279
386  2023-05-20 22:14:59  1814.25  1816.45  1818.20  1814.25  2245.1111
387  2023-05-20 22:29:59  1816.45  1816.22  1818.04  1816.21  1743.5616
388  2023-05-20 22:44:59  1816.21  1816.35  1816.36  1815.16   631.5277
389  2023-05-20 22:59:59  1816.35  1816.54  1816.55  1816.35    71.5842

[390 rows x 6 columns]


In [9]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df, kd_dir):

# ema
    for i in (8, 18, 38):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)

# kd
    df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
    df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
    df['kd_cross'] = check_cross(df, kd_dir)


In [10]:
def conditions(df):

    for index, row in df.iterrows():
        # c1
        df['c1'] = df['kd_cross']
        # c2
        df['c2'] = df['Close'] >= df['ema_8']
        # c3
        df['c3'] = df['ema_8'] >= df['ema_18']
        # c4
        df['c4'] = df['ema_18'] >= df['ema_38']

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4, 'signal'] = True
            
    # 下一根進場
    df['open_entry'] = False

In [11]:
def cancel_orders():
    
    open_orders = client.futures_get_open_orders()
    
    # Cancel all open orders
    if open_orders:
        # Cancel all open orders
        for order in open_orders:
            if order['positionSide'] == 'LONG':  
                cancel_response = client.futures_cancel_order(symbol=order['symbol'], orderId=order['orderId'])
                print(f"Canceled order: {order['symbol']} - {order['orderId']}")
    else:
        # Continue with the rest of the code
        print("No open LONG orders found.")
        time.sleep(1)  # Sleep for 1 second to avoid API rate limit

In [12]:
def enter_position(df):
        
    #-----Calculate entry price-----#
    close_val = df['Close']
    atr_val = df['atr']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = close_val.loc[close_val.index[-2]] - sl_atr * atr_val.loc[atr_val.index[-2]]
    df.loc[df.index[-1], 'take_profit'] = close_val.loc[close_val.index[-2]] + tp_atr * atr_val.loc[atr_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.1
    entry_p = round(df.loc[df.index[-1], 'entry_p'], 2)
    stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'], 2)
    take_profit_p = round(df.loc[df.index[-1], 'take_profit'], 2)
    
    cancel_orders()

    try:
        order = client.futures_create_order(
            symbol=symbol.upper(),
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG',
        )
        print('Order created successfully.')       
        
        take_profit_order = client.futures_create_order(
            symbol=symbol.upper(),
            side='SELL',
            type='TAKE_PROFIT_MARKET',
            stopPrice=take_profit_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'LONG'
        )
        winsound.Beep(frequency, duration)
        print('Take profit created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')
        
    print(str(symbol.upper()) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))

    return df

In [13]:
def check_sl(df, current_k):
    
    # 檢查前一根 Close 是否低於 SL，若低於，即刻停損
    
    last_signal_row = df.loc[df['signal'].eq(True) & df['Close_Time'].ne(current_k)].tail(1)
    if not last_signal_row.empty:
        current_sl = round(df.loc[last_signal_row.index[0], 'Close'] - df.loc[last_signal_row.index[0], 'atr'] * sl_atr, 2)
        current_tp = round(df.loc[last_signal_row.index[0], 'Close'] + df.loc[last_signal_row.index[0], 'atr'] * tp_atr, 2)

    print(f'Current SL at {current_sl}')
    print(f'Current TP at {current_tp}')
    
    # 確定停損
    if df.iloc[df.index[-2]]['Close'] <= current_sl:
        
        print('Last candle closed below SL, try create SL Order')
        
        cancel_orders()

        # 如果目前價格高於停損
        ticker = client.futures_symbol_ticker(symbol=symbol.upper())

        mark_price = round(float(ticker['price']), 2)
        
        stop_price = current_sl
        if current_sl > mark_price:
            stop_price = mark_price
        
        try:

            stop_loss_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='STOP_MARKET',
                stopPrice=stop_price,
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
        
            print(f'Stop loss created successfully at: {current_sl}')
            winsound.Beep(frequency, duration)
        
        except Exception as e:
            
            print(f'Error creating SL order: {e}')
          

In [14]:
import IPython

def restart_and_run_all():
    # Restart the kernel
    IPython.Application.instance().kernel.do_shutdown(True)

    # Re-run all code cells
    IPython.get_ipython().run_line_magic('run', '-i 000_007_LONG_kd&ema&atr_15m.ipynb')


In [15]:
from IPython.display import clear_output

line_count = 0
max_lines = 1
current_k = 0
restart_countdown = 2 * 60 * 60 # restart in 2 hours

def console_log(df):
    global line_count, max_lines, restart_countdown
    try:
        df = df.reset_index(drop=True)
        df = df.round(2)
        print('----------------------------------------------------------------------------')
        print(f"{str(df.loc[df.index[-1], 'Close_Time'])} at {str(df.loc[df.index[-1], 'Close'])}")
        print()
        print(df[['Close_Time', 'Open', 'Close', 'High', 'Low', 'Volume', 'ema_8', 'ema_18', 'slow_k', 'slow_d', 'c1', 'c2', 'c3', 'c4', 'signal', 'open_entry']].tail())
        print('----------------------------------------------------------------------------')

        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Side':position['positionSide'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
                print()

        
        restart_countdown -= 1
        print(f'{restart_countdown} iterations until restart')

        if restart_countdown <= 100:
            restart_and_run_all()
            
    except Exception as e:
        print(f'Error UPDATING info: {e}')    

    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [16]:
def check_time(df):
    global current_k
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:

            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]

            if df.iloc[df.index[-2]]['signal'] == True:

                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'open_entry'] = True
                enter_position(df)

                print(df.tail(2))

    except Exception as e:
        print(f'Error creating ENTRY order: {e}')

In [17]:
def run():
    global current_k, restart_countdown

    while True:
        
        try:
            df = get_historical(symbol, interval, start_time, end_time, step)
            indicators(df, kd_dir)
            conditions(df)
            check_time(df)

            # 檢查停損
            try:
                positions_info = client.futures_account()['positions']
                long_positions = [p for p in positions_info if p['positionSide'] == 'LONG' and float(p['positionAmt']) != 0]

                if long_positions:

                    check_sl(df, current_k)

                else:
                    print('No LONG position. No SL')

            except Exception as e:
                print(f'Error checking SL: {e}')

            # 更新狀態
            console_log(df)
            
            time.sleep(2)
            
        except ConnectionError as e:
            print("Connection error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)

In [18]:
sl_atr = 1
tp_atr = 12
kd_dir = 'Any'

In [19]:
run()   

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))

In [ ]:
# kd cross，close > ema 8, ema 8 > ema 18, ema 18 > 38, take profit atr, stop loss atr